## Optimization of an Individual's Diet
- Inputs for this notebook are supplied by the User_Inputs.ipynb jupyter notebook dropdown widgets and from the dataframe that is the result of running Build_Optimization_Database.ipynb and then Foods_to_Include.ipynb notebooks. Foods_to_Include prepares the database for linear optimization.

- If you want to add a new impact category to this function, see runopt() function for where to add this and add a new column with the appropriate impacts.

In [2]:
# Packages that are necessary
import numpy as np
import pandas as pd
import pickle
import xlrd
import pulp as pulp
from pulp import *

In [1]:
# this is a function to find necessary input files regardless of where they are stored.
def findfiles(filename,sheet):
    top = %pwd
    for root, dirs, files in os.walk(top):
        for name in files:
            if name==filename:
                try:
                    file = pd.read_pickle(os.path.join(root,filename))
                except:
                    if len(sheet)>0:
                        file = pd.read_excel(os.path.join(root,filename),sheet_name = sheet)
                    else:
                        file = pd.read_excel(os.path.join(root,filename))
    return file

In [ ]:
def filter(foodgroup,variable):
    # this is a function to filter out food groups from the database, and is used in the optimization code below.
    filtered = {k:v for (k,v) in variable.items() if any('_'+q in k for q in foodgroup)}
    return filtered

###### These functions set the minimum mass of each type of food item allowed for the optimization. Depends on food type and number of days the diet is optimized for. This is to eliminate suggestions of serving sizes of, for example, 1 gram of an apple as being part of an optimizated diet.

In [ ]:
def fruitlimits(df,numdays):
    fruit_total = 250*numdays # grams per day
    fruit_groups =('FAT','FAT')
    fruit_serv_min = 75
    return fruit_total,fruit_groups,fruit_serv_min

In [ ]:
def veglimits(df,numdays):
    vegetable_total = 360*numdays # grams per day
    vegetable_groups = ('DGR','DGC')
    vegetable_serv_min = 50 ## largest serving size of vegetables
    return vegetable_total,vegetable_groups,vegetable_serv_min

In [ ]:
def legumelimits(df,numdays):
    legume_total = 60*numdays # grams per day
    legume_serv_min = 50 ## largest serving size of legumes
    legume_groups = ('DBR','DBC','DFR','DFC')
    return legume_total,legume_groups,legume_serv_min

In [ ]:
def grainlimits(df,numdays):
    grains_total = 125*numdays # grams per day
    grains_groups = ('AA_wg','AC_wg','AD_wg','AI_wg','AF_wg')
    grains_num = 3
    pasta_rice_pot_serve_min = 50
    pasta_rice_group = ['AC','AC_wg','DAR','DAC']
    cereal_grain_bread_serve_min = 30
    cereal_grain_bread_group = ['AI','AI_wg','AA','AA_wg','AF','AF_wg','AD','AD_wg','AC','AC_wg']
    return (grains_total,grains_groups,pasta_rice_pot_serve_min,pasta_rice_group,cereal_grain_bread_serve_min,
        cereal_grain_bread_group)

In [ ]:
def nutslimits(df,numdays):
    nuts_total = 21*numdays # grams per day
    nuts_serv_min = 10 ## largest serving size of nuts
    nuts_groups = ('GAT','GAT')  
    return nuts_total,nuts_groups,nuts_serv_min

In [ ]:
def milklimits(df,numdays):
    milk_total = 435*numdays # grams per day
    milk_serv_min = 50
    milk_groups = ('BAE','BAH','BAK')
    return milk_total,milk_groups,milk_serv_min
    

In [ ]:
def cheeselimits(df,numdays):
    soft_cheese_serve_min = 50
    soft_group =['BLS']
    med_cheese_serve_min = 40
    med_group =['BLM']
    hard_cheese_serve_min = 20
    hard_group =['BLH']
    fresh_cheese_serve_min = 50
    fresh_group =['BLF']
    yogurt_serve_min = 50
    yogurt_group =['BN']
    return (soft_cheese_serve_min,soft_group,med_cheese_serve_min,med_group,hard_cheese_serve_min,hard_group,
       fresh_cheese_serve_min,fresh_group, yogurt_serve_min,yogurt_group)

In [ ]:
def milksublimits(df,numdays):
    milksub_serve_min = 50
    milksub_group =['BAV']
    yogsub_serve_min = 50
    yogsub_group = ['BNV']
    return milksub_serve_min,milksub_group,yogsub_serve_min,yogsub_group

In [ ]:
def veganlimits(df,numdays):
    vegan_serve_min = 100
    vegan_group =['VEG']
    return vegan_serve_min,vegan_group

In [ ]:
def meatlimits(df,numdays):
    fish_serve_min = 50
    fish_groups = df[df['Group'].str.startswith('J')]['Group'].tolist()
    
    red_meat_max = 22.5*numdays
    red_meat_groups = ['MBGR','MBGC','MACR','MACC','MAER','MECC','MECR','MEER','MEEC','MAIC','MAIR','MAE',\
                       'MAIR','MAIC']
    red_serve_min = 22.5

    whitemeat_serve_min = 50
    white_meat_groups = ['MCOR','MCOC','MCAR','MCAC','MAGR','MAGC','MAHR','MAHC']
    
    processed_min = 2
    processed_total = 2*numdays # grams per day
    processed_groups = ['MI','MAAC','MAAR']

    oil_serve_min = 14
    oil_num = 2
    oil_min = oil_serve_min-oil_serve_min*0.1
    oil_max = oil_serve_min+oil_serve_min*0.1
    oil_groups = df[(df['Group'].str.startswith('O'))|(df['Group']=='BTM')|(df['Group']=='BJC')]['Group'].tolist()

    egg_serve_min = 60 # this assumes one large egg at 60 grams each 
    egg_groups = df[df['Group']=='CA']['Group'].tolist()
    return (fish_serve_min,fish_groups,red_meat_max,red_meat_groups,red_serve_min,whitemeat_serve_min,
        white_meat_groups,processed_total,processed_min,processed_groups,oil_serve_min,oil_min,oil_max,oil_groups,
            egg_serve_min,egg_groups)

##### this accesses the nutrient database to determine the necessary nutrients for each age, gender, etc and returns a list of the minimum necessary nutrient values required to be met in the optimization

In [ ]:
def nutrient(gender,age,numdays):
    df_nut = findfiles('7f_to7_nutrients.xlsx','')
    names = df_nut.columns.tolist()[2:]
    mass = [i.split(')',1)[0].split('(')[1] for i in df_nut.loc[0].tolist()[2:]]
    ## calculate the b inequality values for optimization based on user input   
    if (gender==0 and age<=30):
        nut_limits = [i for i in df_nut.loc[10]][2:]
    elif (gender==0 and age>30 and age<51):   
        nut_limits = [i for i in df_nut.loc[11]][2:]
    elif (gender==0 and age>51 and age<70):   
        nut_limits = [i for i in df_nut.loc[12]][2:]
    elif (gender==0 and age>70):   
        nut_limits = [i for i in df_nut.loc[13]][2:]
    elif (gender==1 and age<=30):
        nut_limits = [i for i in df_nut.loc[17]][2:]
    elif (gender==1 and age>30 and age<51):   
        nut_limits = [i for i in df_nut.loc[18]][2:]
    elif (gender==1 and age>51 and age<70):   
        nut_limits = [i for i in df_nut.loc[19]][2:]
    elif (gender==1 and age>70): 
        nut_limits = [i for i in df_nut.loc[20]][2:]
    else: ## default woman between 31 and 50
        nut_limits = [i for i in df_nut.loc[18]][2:]
    
    return [(j,round(i*numdays,2),m) for i,j,m in zip(nut_limits,names,mass)]

##### this calculates the necessary energy for an individual based on parameters duch as activity levels, height, weight, age, and gender

In [ ]:
def personal(gender,weight,height,age,actlevel,numdays):
    age = age
    weight = weight ## in kg
    height = height ## in cm   
    activity = PAL(actlevel)
    if gender == 0:
        kcal = (10*weight + 6.25*height - 5*age - 161)*activity*numdays
    else:
        kcal = (10*weight + 6.25*height - 5*age + 5)*activity*numdays  
    nut_limits = nutrient(gender,age,numdays)    
    return kcal,nut_limits    

In [ ]:
# The PAL of a subject can be classified in three categories as defined by the last FAO/WHO/UNU 
# expert consultation on human energy requirements (2004). The physical activity for sedentary and 
# light activity lifestyles ranges between 1.40 and 1.69, for moderately active or active lifestyles 
# between 1.70 and 1.99, and for vigorously active lifestyles between 2.00 and 2.40.
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3636460/

In [ ]:
def PAL(lifestyle):
    if lifestyle == 'sedentary':
        activity = 1.4
    elif lifestyle == 'moderate':
        activity = 1.7
    elif lifestyle == 'vigorous':
        activity = 2.0
    else: ## default
        activity = 1.4
    return activity

##### this is the function below to run the optimization itself and incorporates all the functions above
- Input name into function that calculates the diet = how to see what that value is based on inputs here. These inputs are collected in the run_tool() function in User_Inputs.ipynb and are automatically used as inputs into the Diet_Optimization_Tool.ipynb.
- df = df_foods (the database of all foods, impacts, nutrients
- impact = imp.value (chosen impact for optimization)
- preference = diettype.value (diet type,vegetarian, omnivorous, etc.)
- supplement = supp[sup.value] (whether a supplement is ok or not)
- gender = gender[gen.value] (biological sex)
- weight = personweight.value (individual's weight)
- height = height.value (individual's height)
- age = age.value (individual's age)
- actlevel = act.value (individual's activity)
- numdays = days.value (number of days for the diet)
- relax = relaxfactor (factor indicating the upper limit on allowed daily mass 2 is one serving size per day)
- include = selectedfoods (dictionary of all mandatory foods based on preferences)
- vitd = vitd[vitamind.value] (should Vitamin D be included as a constraint or not)

In [3]:
def runopt(df,impact,preference,supplement,gender,weight,height,age,actlevel,numdays,relax,include,vitd):
# FINAL OUTPUT IS A DATABASE CONTAINING A RECOMMENDED DIET FOR AN INDIVIDUAL    
    
    global df_final
    global status

############## RESULTS OF FUNCTIONS ABOVE ########################################################################
# set serving size limits, nutrient needs, etc.
    kcal,nut_limits = personal(gender,weight,height,age,actlevel,numdays) # get the right kcal and nutrients
    fruit_total, fruit_groups, fruit_serv_min = fruitlimits(df,numdays)
    vegetable_total,vegetable_groups,vegetable_serv_min = veglimits(df,numdays)
    legume_total,legume_groups,legume_serv_min = legumelimits(df,numdays)
    grains_total,grains_groups,pasta_rice_pot_serve_min,pasta_rice_group,cereal_grain_bread_serve_min,\
            cereal_grain_bread_group = grainlimits(df,numdays)
    nuts_total,nuts_groups,nuts_serv_min = nutslimits(df,numdays)
    milk_total,milk_groups,milk_serv_min = milklimits(df,numdays)
    soft_cheese_serve_min,soft_group,med_cheese_serve_min,med_group,hard_cheese_serve_min,hard_group,\
           fresh_cheese_serve_min,fresh_group, yogurt_serve_min,yogurt_group = cheeselimits(df,numdays)
    fish_serve_min,fish_groups,red_meat_max,red_meat_groups,red_serve_min,whitemeat_serve_min,\
            white_meat_groups,processed_total,processed_min,processed_groups,oil_serve_min,oil_min,oil_max,oil_groups,\
                egg_serve_min,egg_groups = meatlimits(df,numdays)  
    vegan_serve_min,vegan_group = veganlimits(df,numdays)
    milksub_serve_min,milksub_group,yogsub_serve_min,yogsub_group = milksublimits(df,numdays)
    PUFA = 0.11 # more than 11% of energy should come from polyunsaturated fatty acids
    transFA = 0.005 #less than 0.5% of kcal per day should come from trans fats
    omega3 = 250*numdays # mg per day 
    nutrient_source = ['Calcium (mg)','Copper (mg)', 'Iodine (µg)', 'Iron (mg)', 'Magnesium (mg)','Manganese (mg)',
        'Phosphorus (mg)', 'Selenium (µg)','Zinc (mg)','Potassium (mg)', 'Sodium (mg)','Chloride (mg)', 
        'Retinol Equivalent (µg)','Vitamin C (mg)', 'Vitamin D (µg)', 'Vitamin E (mg)', 'Vitamin K1 (µg)', 'Thiamin (mg)',
        'Riboflavin (mg)', 'Niacin equivalent (mg)', 'Vitamin B6 (mg)', 'Folate (µg)', 'Vitamin B12 (µg)',
        'Pantothenate (mg)','Biotin (µg)', 'Carbohydrate (g)','AOAC fibre (g)', 'Fat (g)','Protein (g)']

##################### SET UP OPTIMIZATION PROBLEM ####################################################################
    prob = LpProblem('Optimization',LpMinimize)
        # Create a list of the foods 
    foodgroups=list(df['Food Name'])
    food_status = LpVariable.dicts("food_status",(foodgroup for foodgroup in foodgroups),cat='Binary')        
    x = {foodamount:LpVariable(foodamount, lowBound=0) for foodamount in foodgroups}  

    # the following code sets the objective function - i.e. what impact do we want to minimize for
    # one can easily add other impacts to the optimization by adding the corresponding impacts as a new column to the 
    # df
    if impact == 'GHG':
        impactsnew = dict(zip(foodgroups,df['optimization_value_GHG_1_tradeopt'])) # objective 1
        prob += lpSum([impactsnew[i]*x[i]for i in foodgroups]), "Total Impacts per person"
    elif impact == 'BIO':
        impactsnew = dict(zip(foodgroups,df['optimization_value_BIO_1_tradeopt'])) # objective 2
        prob += lpSum([impactsnew[i]*x[i]for i in foodgroups]), "Total Impacts per person"
    
    #%%%%%%%%%%%% new impacts column %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    elif impact == 'NEW IMPACT TYPE':
        impactsnew = dict(zip(foodgroups,df['NEW_IMPACT_VALUE_COLUMN'])) # objective 2
        prob += lpSum([impactsnew[i]*x[i]for i in foodgroups]), "Total NEW Impacts per person"    
    #%%%%%%%%%%%% new impacts column %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        
    else: # considering both GHG and BIO - other impacts can be added here.       
        weighting = 0.5
        ########### these are needed to normalize the values for the multi-objective optimization #################
        min_value_GHG = 2
        min_value_bio = 0.5e-18
        max_value_GHG =  min_value_GHG*10
        max_value_bio = min_value_bio*100
        #############################################################
        opt1 = dict(zip(foodgroups,df['optimization_value_GHG_1_tradeopt']))
        opt2 = dict(zip(foodgroups,df['bio_GHGopt_value1opt']))
        impactGHG = lpSum([opt1[i]*x[i]for i in foodgroups])
        impactBIO = lpSum([opt2[i]*x[i]for i in foodgroups])
        prob += (weighting)*(impactGHG-min_value_GHG)/(max_value_GHG-min_value_GHG)+\
                    (1-weighting)*(impactBIO-min_value_bio)/(max_value_bio-min_value_bio), "Total Impacts per person"  

    
###   ALL CONSTRAINTS     ##############################################################################

# these criteria must be met in the diet.

####################################### NUTRIENT CONSTRAINTS ############################################
    
    # sodium is excluded from this constraint because it is with the GBD requirements
    # vitamin D is excluded because it is considered separately
    nut_lim_noVD =[i[1] for i in nut_limits if 'Vitamin D' not in i[0] if 'Sodium' not in i[0]]
    nut_source_noVD = [i for i in nutrient_source if 'Vitamin D' not in i if 'Sodium' not in i]
    for nutlimit,nutrient in zip(nut_lim_noVD,nut_source_noVD):
        prob += (lpSum([df[df['Food Name']==i][nutrient]*x[i]for i in foodgroups])>=nutlimit,
                 'Nutrient Constraint: %s%s'%(nutlimit,nutrient))

    # if the user wants to include Vitamin D, this constraint is added. Otherwise it is passed.
    if vitd==0: pass
    else:
        prob += (lpSum([df[df['Food Name']==i]['Vitamin D (µg)']*x[i]for i in foodgroups])>=[i[1] for i in nut_limits if i[0]=='Vitamin D'][0],
                           'Nutrient Constraint: %s%s'%(nutlimit,'Vitamin D (µg)'))
   
    # setting the min kcal contraint
    prob += (lpSum([df[df['Food Name']==i]['Energy (kcal) (kcal)']*x[i]for i in foodgroups])>=kcal,'kcal')

################################ ELASTIC CONSTRAINTS ON MAXIMUM LIMITS ##################################
    
    # include are the food items and masses that the user wants in their diet regardless of impact or health consequences
    # if they selected foods, certain maximum constraints (sodium and TFA) may need to be overwritten in order to find an 
    # optimal solution. Otherwise, the standard constraints (from GBD) are used.
    if len(include)>0:   

        c_LHS_sod = LpAffineExpression([(x[i],df[df['Food Name']==i]['Sodium (mg)'].values[0]) for i in foodgroups])
        elas_con_sod = LpConstraint(e=c_LHS_sod, name='sodium',rhs=2300*numdays)
        c1_elastic = elas_con_sod.makeElasticSubProblem(penalty = 100, proportionFreeBoundList = [.10,.10])
        prob.extend(c1_elastic)
    
        c_LHS_trans = LpAffineExpression([(x[i],df[df['Food Name']==i]['Trans FA_kcal'].values[0]) for i in foodgroups])
        elas_con_trans = LpConstraint(e=c_LHS_trans, name='TFA',rhs=transFA*kcal)
        c2_elastic = elas_con_trans.makeElasticSubProblem(penalty = 100, proportionFreeBoundList = [.10,.10])
        prob.extend(c2_elastic)      
   
    else:
        prob += (lpSum([df[df['Food Name']==i]['Sodium (mg)']*x[i]for i in foodgroups])<=2300*numdays,'sodium')
        prob += (lpSum([df[df['Food Name']==i]['Trans FA_kcal']*x[i] for i in foodgroups])<= transFA*kcal,\
                     'TFA constraints')    
    
######################################### BINARY CONSTRAINTS #########################################################
    
    # this is required to ensure that the optimization doesn't select similar food items to meet constraints.
    # for example, if fortified bran breakfast cereal is selected, the optimization can't also select wheat cereal, 
    # unfortified bran cereal, etc. These constraints were developed based on the Big M method.
    for i,j in zip(foodgroups,food_status):
        prob += food_status[j]*1<=x[i],'Binary Constraint_1: %s'%i # if xi == 0, food status i is allowed
                # to be 0. when xi > 0, food_status i is forced to flip to 1 in order to raise the upper bound to M.
        prob += x[i]<=food_status[j]*100000, 'Binary Constraint_2: %s'%i
        
        
######################################### GBD Dietary Risk CONSTRAINTS ###############################################  
    
    # the GBD sets minimum and maximum limits on certain food groups
    # this makes sure the minimimum amount of fruits, vegetables, legumes, grains, and nuts are consumed
    for limit,group in zip([fruit_total,vegetable_total,legume_total,grains_total,nuts_total],
                   [fruit_groups,vegetable_groups,legume_groups,grains_groups,nuts_groups]):
        prob += (lpSum([filter(group,x)[i] for i in filter(group,x)])>=limit,\
                         'GBD Constraint: %s%s'%(limit,group))
    # this makes sure the maximum amount of processed meats are consumed
    prob += (lpSum([filter(processed_groups,x)[i] for i in filter(processed_groups,x)])<=processed_total,\
                         'GBD Constraint: %s%s'%(processed_total,processed_groups))
    
    # One of the GBD constraints is a minimum milk consumption. If a person is a vegan, this constraint is overwritten,
    # otherwise it is included.
    if (preference == 'none')|(preference == 'vegetarian')|(preference == 'pescatarian'): 
        for limit,group in zip([milk_total],[milk_groups]):
            prob += (lpSum([filter(group,x)[i] for i in filter(group,x)])>=limit,\
                             'GBD Constraint: %s%s'%(limit,group))
    #elif preference == 'vegetarian':
    #    for limit,group in zip([milk_total],[milk_groups]):
    #        prob += (lpSum([filter(group,x)[i] for i in filter(group,x)])>=limit,\
    #                         'GBD Constraint: %s%s'%(limit,group))
    elif (preference == 'vegan')|(preference == 'nomilk'):pass      
    else:pass

    # nutrient constraints from GBD
    # other GBD nutrient constraints are included in the nutrient limit constraints
    prob += (lpSum([df[df['Food Name']==i]['Omega3_all']*x[i] for i in foodgroups])>= omega3,\
                     'Omega3 constraints')
    prob += (lpSum([df[df['Food Name']==i]['PUFA_kcal_POLY_DB']*x[i] for i in foodgroups])>= PUFA*kcal,\
                     'PUFA constraints') 

################################# THESE ARE THE ITEMS THAT MUST BE INCLUDED BASED ON USER PREFERENCES  ###############
    
    for key,value in include.items():
        prob += (lpSum({k:v for (k,v) in x.items() if value['names'] in k})>=value['mass'])
    
##################################  SERVING SIZE LIMITS #############################################################
 
##################################     minimum limits     #############################################################
    # this sets minimum sizes for each type of food item, to prevent a suggestion of, food example, one gram of a 
    # certain food item. Minimum serving sizes are set with the functions above, and are different for each 
    # food group.
    
    # fruits, vegetables, legumes, nutes/seeds, milk, min limits
    for serve,i in zip([fruit_serv_min,vegetable_serv_min,legume_serv_min,nuts_serv_min,milk_serv_min],
                    [fruit_groups,vegetable_groups,legume_groups,nuts_groups,milk_groups]):
        for k,l in zip(filter(i,x),filter(i,food_status)):
            prob += filter(i,x)[k] >= serve * filter(i,food_status)[l],\
                        'Min Serving Size Limits: %s%s%s'%(serve,i,k)

    # cheese and yogurt min limits
    for serve,i in zip([soft_cheese_serve_min,med_cheese_serve_min,hard_cheese_serve_min,
                                    fresh_cheese_serve_min,yogurt_serve_min],
                                   [soft_group,med_group,hard_group,fresh_group,yogurt_group]):
        for k,l in zip(filter(i,x),filter(i,food_status)):
            prob += filter(i,x)[k] >= serve * filter(i,food_status)[l],\
            'Min Serving Size Limits: %s%s%s'%(serve,i,k)

    # vegan milk and vegan yogurt min limits
    for serve,i in zip([milksub_serve_min,yogsub_serve_min,vegan_serve_min],
                                   [milksub_group,yogsub_group,vegan_group]):
        for k,l in zip(filter(i,x),filter(i,food_status)):
            prob += filter(i,x)[k] >= serve * filter(i,food_status)[l],\
                        'Min Serving Size Limits: %s%s%s'%(serve,i,k)        
    
    # grain product min limits
    for serve,i in zip([pasta_rice_pot_serve_min,cereal_grain_bread_serve_min],
                                   [pasta_rice_group,cereal_grain_bread_group]):
        for k,l in zip(filter(i,x),filter(i,food_status)):
            prob += filter(i,x)[k] >= serve * filter(i,food_status)[l],\
                        'Min Serving Size Limits: %s%s%s'%(serve,i,k) 
    
    # meat,egg, oil, fish product min limits      
    for serve,i in zip([fish_serve_min,red_serve_min,whitemeat_serve_min,oil_serve_min,egg_serve_min],
                                   [fish_groups,red_meat_groups,white_meat_groups,oil_groups,egg_groups]):
        for k,l in zip(filter(i,x),filter(i,food_status)):
            prob += filter(i,x)[k] >= serve * filter(i,food_status)[l],\
                        'Min Serving Size Limits: %s%s%s'%(serve,i,k)
    

    
##################################     maximum limits     #############################################################    
    
    # these are constraints to limit the mass of each food item that can be selected in the optimization
    # the maximum limits are a function of the minimum amounts (minsize) (2*min serving amount == 1 serving size)
    # the default maximum is one serving size of each food item per day of diet (i.e. 7 day diet allows 
    # 7 serving sizes in that total diet, to be consumed as desired)
    # relax is how much you are increasing the serving size. Default is 2 (for 1 serving size).
    # for vegetarian and vegan diets without supplementation, a maximum of 1 serving size per day isn't enough to
    # obtain adequate nutrients, therefore if an infeasible solution is found the code is automaticall run to increase
    # the allowed serving sizes. (this happens in User_Inputs.ipynb notebook in function 'run_tool', which runs this code)
    
    groups = [fruit_groups,vegetable_groups,pasta_rice_group,cereal_grain_bread_group,legume_groups,nuts_groups,
              fish_groups,white_meat_groups,oil_groups,egg_groups,red_meat_groups,processed_groups,
            vegan_group,milksub_group,yogsub_group,yogurt_group,fresh_group,soft_group,med_group,hard_group]
    size = [fruit_serv_min,vegetable_serv_min,pasta_rice_pot_serve_min,cereal_grain_bread_serve_min,
           legume_serv_min,nuts_serv_min,fish_serve_min,whitemeat_serve_min,oil_serve_min,egg_serve_min,red_serve_min,
            processed_min,vegan_serve_min,milksub_serve_min,yogsub_serve_min,yogurt_serve_min,fresh_cheese_serve_min,
            soft_cheese_serve_min,med_cheese_serve_min,hard_cheese_serve_min]
    
    for foodgroup,minsize in zip(groups,size):
        for k,l in zip(filter(foodgroup,x),filter(foodgroup,food_status)):
            # this is necessary to not include limits to the personal food items that are desired - there are no maximums 
            # for those items in which the user states how much they want.
            if k not in [v['names'] for k,v in include.items()]:
                prob += filter(foodgroup,x)[k] <= relax*minsize*numdays*\
                          filter(foodgroup,food_status)[l]
            else:pass
                #prob += filter(foodgroup,x)[k] <= [v1['mass']*1.1 for k1,v1 in include.items()\
                #            if k in v1['names']][0]*filter(foodgroup,food_status)[l]      
    
    for k,l in zip(filter(milk_groups,x),filter(milk_groups,food_status)):
        prob += filter(milk_groups,x)[k] <= 1000*numdays*\
                        filter(milk_groups,food_status)[l],('Max Limits_BAX: %s%s'%(k,l))

    for cheese_type,min_serve in zip([fresh_group,soft_group,med_group,hard_group,yogurt_group],\
                            [fresh_cheese_serve_min,soft_cheese_serve_min,med_cheese_serve_min,\
                             hard_cheese_serve_min,yogurt_serve_min]):      
        for k,l in zip(filter(cheese_type,x),filter(cheese_type,food_status)):
            prob += filter(cheese_type,x)[k] <= relax*min_serve*numdays*\
                            filter(cheese_type,food_status)[l],('Max Limits_BXX: %s%s%s%s'%(k,l,cheese_type,min_serve))

    #(4 grams per clove, 5 cloves per day)  
    prob += (lpSum({k:v for (k,v) in x.items() if 'garlic' in k})<=4*5*numdays*relax),('Max Limits garlic')

    
##################################  LIMITING SELECTION OF SIMILAR ITEMS   #############################################
    
    # by default, can't choose same food item more than 1 time (i.e. fresh and canned and frozen peaches) 
    # to meet constraints (exception for can select more than 2 oils) ------ UNLESS --------
    # user food preferences include similar items in a particular food category, then this constraint is overridden
    # and similar items in a food category will be included in the optimal diet as requested.
    
    for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if 'FAT' in k})):
        subset0 = {k:v for (k,v) in food_status.items() if k.startswith(i) and 'FAT' in k}
        if len([value['names'] for key,value in include.items() if 'FAT' in value['names']])>=1:pass           
        else:prob += lpSum(subset0)<=1,'Food Overlap Constraint_FAT%s'%i 
    for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if 'GAT' in k})):
        subset5 = {k:v for (k,v) in food_status.items() if k.startswith(i) and 'GAT' in k}
        if len([value['names'] for key,value in include.items() if 'GAT' in value['names']])>=1:pass             
        else:prob += lpSum(subset5)<=1,'Food Overlap Constraint_GAT%s'%i
    for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_AI' in k})):
        subset6 = {k:v for (k,v) in food_status.items() if i in k}
        if len([value['names'] for key,value in include.items() if '_AI' in value['names']])>=1:pass
        else:prob += lpSum(subset6)<=1,'Food Overlap Constraint_AI%s'%i 
    for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_DF' in k})):
        subset7 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_DF',i])}
        if len([value['names'] for key,value in include.items() if '_DF' in value['names']])>=1:pass
        else:prob += lpSum(subset7)<=1,'Food Overlap Constraint_DF%s'%i         
    for i in list(set({k.split(',')[1] for (k,v) in food_status.items() if '_DB' in k})):
        subset8 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_DB',i])}
        if len([value['names'] for key,value in include.items() if '_DB' in value['names']])>=1:pass
        else:prob += lpSum(subset8)<=1,'Food Overlap Constraint_DB%s'%i       
    for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_DG' in k})):
        subset9 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_DG',i])}
        if len([value['names'] for key,value in include.items() if '_DG' in value['names']])>=1:pass
        else: prob += lpSum(subset9)<=1,'Food Overlap Constraint_DG%s'%i
    for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_DA' in k})):
        subset15 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_DA',i])}
        if len([value['names'] for key,value in include.items() if '_DA' in value['names']])>=1:pass
        else: prob += lpSum(subset15)<=1,'Food Overlap Constraint_DA%s'%i        
    for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_J' in k})):
        subset10 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_J',i])}
        if len([value['names'] for key,value in include.items() if '_J' in value['names']])>=1:pass
        else:prob += lpSum(subset10)<=1,'Food Overlap Constraint_J%s'%i 
    for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_BA' in k})):
        subset11 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_BA',i])}
        if len([value['names'] for key,value in include.items() if '_BA' in value['names']])>=1:pass
        else: prob += lpSum(subset11)<=1,'Food Overlap Constraint_BA%s'%i          
    for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_CA' in k})):
        subset12 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_CA',i])}
        if len([value['names'] for key,value in include.items() if '_CA' in value['names']])>=1:pass
        else:prob += lpSum(subset12)<=1,'Food Overlap Constraint_CA%s%s'%(i,k)                
    for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_BAV' in k})):
        subset13 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_BAV',i])}
        if len([value['names'] for key,value in include.items() if '_BAV' in value['names']])>=1:pass
        else:prob += lpSum(subset13)<=1,'Food Overlap Constraint_BAV%s%s'%(i,k)                             
    for i in list(set({k.split(',')[0] for (k,v) in food_status.items() if '_AF' in k})):
        subset14 = {k:v for (k,v) in food_status.items() if all(x in k for x in ['_AF',i])}
        if len([value['names'] for key,value in include.items() if '_AF' in value['names']])>=1:pass
        else:prob += lpSum(subset14)<=1,'Food Overlap Constraint_AF%s%s'%(i,k)            

    subset1 = {k:v for (k,v) in food_status.items() if '_BTM' in k}
    if len([value['names'] for key,value in include.items() if '_BTM' in value['names']])>=1:pass    
    else: prob += lpSum(subset1)<=1,'Food Overlap Constraint%s'%'BTM'
    subset2 = {k:v for (k,v) in food_status.items() if '_OA' in k}
    if len([value['names'] for key,value in include.items() if '_OA' in value['names']])>=1:pass
    else: prob += lpSum(subset2)<=1,'Food Overlap Constraint%s'%'OA'
    subset3 = {k:v for (k,v) in food_status.items() if '_OC' in k}
    if len([value['names'] for key,value in include.items() if '_OC' in value['names']])>=1:pass
    else: prob += lpSum(subset3)<=2,'Food Overlap Constraint%s'%'OC'  


######################################### SOLVE OPTIMIZATION PROBLEM ################################################

    prob.writeLP('Optimization.lp')
    prob.solve()
    status = prob.solve()

######################################### SAVE RESULTS ##############################################################    
    
    # if the problem is solved, follow this path
    if LpStatus[prob.status]=='Optimal':
        print ('We have found an optimized diet.')
        results = []
                    # Each of the variables is printed with its resolved optimum value
        for v in prob.variables():
            re=[v.name,v.varValue]
            results.append(re)
        results_food = [i for i in results if 'food_status' not in i[0]] # REMOVE BINARY VARIABLE RESULTS
        results_food = [i for i in results_food if 'dummy' not in i[0]]
        results_food_dict = {item[0]:item[1] for item in results_food}
        constraints = []
        for name, c in prob.constraints.items():
    #print (name,':', c.pi,'slack:', c.slack) - constraint name, shadow price, slack
            x = [name,c.pi,c.slack]
            constraints.append(x)
        df['Food Name'] = [i.replace(' ','_') for i in df['Food Name']]
        df['Grams'] = df['Food Name'].map(results_food_dict)
        df_final = df[['Food Name','Grams','optimization_country_GHG_1_trade','optimization_value_GHG_1_trade',
            'bio_GHGopt_value1', 'optimization_value_BIO_1_trade','optimization_country_BIO_1_trade',
                       'GHG_bioopt_value1','Food Item']]   
    
    # if the problem is not feasible after several tries, this is the generic diet until the problem can be resolved.
    else:
        columns = ['Food Name','Grams','optimization_country_GHG_1_trade','optimization_value_GHG_1_trade',
            'bio_GHGopt_value1', 'optimization_value_BIO_1_trade','optimization_country_BIO_1_trade',
                       'GHG_bioopt_value1','Food Item']
            
        data = [['apples,_eating,_raw,_flesh_and_skin,_weighed_with_core_FAT',  700.0,  'NA',  0.00020527110400302525,
          1.0053053414053236e-18,  1.2749938858420666e-19,  'NA',  0.0003487290738592284,  'apples'],
         ['sunflower_seeds,_toasted_GAT',  140.0,  'NA',  0.0006391992011490255,  2.871373164031176e-18,  2.603193330279975e-18,
          'NA',  0.000925380683201635,  'sunflower seeds, toasted'], ['beans,_broad,_dried,_raw,_canned_DBR',  443.06589,
          'NA',  0.00033183518827853875,  3.850140293241735e-17,  5.920335757669555e-19,  'NA',  0.0005803103584835859,  'broad beans, dried, canned'],
         ['beans,_soya,_dried,_raw,_canned_DBR',  700.0,  'NA',  0.0005993640433093169,  1.971703441696136e-18,  1.1444609674191097e-18,
          'NA',  0.003874829372820653,  'soya beans, dried, canned'], ['bread,_wholemeal,_average_AF_wg',  223.60573,
          'NA',  0.00047726285730036703,  1.3212286302452188e-18,  1.4393768486777955e-19,  'NA',  0.0010297962125251638,
          'bread, wholemeal, average'], ['cereal,_malted_flake,_fortified_AI',  420.0,  'NA',  0.000749732818241673,
          1.7716474814651797e-18,  4.775523635945825e-19,  'NA',  0.0014880675400756398,  'cereal, malted flake, fortified'],
         ['noodles,_egg,_dried,_raw_AD',  420.0,  'NA',  0.0005874867788047805,  1.5164101324405352e-18,  8.235970191516377e-19,
          'NA',  0.0008140617088481893,  'noodles, egg, dried, raw'], ['pasta,_wholewheat,_spaghetti,_dried,_raw_AD_wg',
          404.21196,  'NA',  0.0005874867788047805,  1.5164101324405352e-18,  8.235970191516377e-19,  'NA',\
        0.0008140617088481892,  'pasta, wholewheat, spaghetti, dried, raw'],
         ['carrots,_young,_raw_DGR',
          700.0,  'NA',  0.0001804894786328296,  2.0768486167720712e-19,  1.2735688713920142e-19,
          'NA',  0.0001943243270789832,  'carrots'], ['curly_kale,_raw_DGR',  337.01335,
          'NA',  0.0002219144602161762,  3.394532437659784e-18,  3.394532437659784e-18,  'NA',  0.0002219144602161762,  'curly kale'],
         ['pumpkin_seeds_GAT',  140.0,  'NA',  0.00027447907742553733,  1.512744193409618e-18,  1.512744193409618e-18,  'IT',
          0.00027447907742553733,  'pumpkin seeds'], ['lettuce,_average,_raw_DGR',  82.986645,  'NA',  0.00018272919937603,
          3.929668780509031e-19,  3.472990515619952e-19,  'NA',  0.0002544615371582102,  'lettuce'], ['rocket,_raw_DGR',
          700.0,  'NA',  0.0001548552537085,  3.330227780092399e-19,  2.9432123013728405e-19,  'NA',  0.0002156453704730595,
          'rocket'], ['spring_greens,_raw_DGR',  700.0,  'NA',  0.00017963209430186,  3.8630642249071825e-19,  3.414126269592495e-19,
          'NA',  0.00025014862974874903,  'spring greens'], ['milk,_whole,_pasteurised,_average_BAK',  3045.0,
          'NA',  0.0019283754712295452,  3.479675786746047e-18,  2.5511330401079282e-18,  'NA',
          0.001949774115522527,  'whole milk pasteurised, average'], ['muesli,_swiss_style,_unfortified_AI_wg',
          420.0,  'NA',  0.000318877817790194,  4.535923319752412e-19,  4.535923319752412e-19,  'NA',
          0.000318877817790194,  'muesli, swiss style, unfortified'], ['sesame_seeds_GAT',  115.4351,  'NA',
          0.0010386950775170714,  2.8730021975579434e-18,  2.8730021975579434e-18,  'EG',  0.0010386950775170714,
          'sesame seeds'], ['pears,_average,_flesh_and_skin,_raw,_weighed_with_core_and_stalk_FAT',  1050.0,
          'NA',  9.166573550047015e-05,  2.6079933219704155e-19,  6.091398834291173e-20,  'NA',  0.0005208825832804054,  'pears'],
         ['potatoes,_old,_baked,_flesh_and_skin_DAC',  700.0,  'NA',  0.00014660025207831106,  2.21025403385686e-19,
          1.2326436852829227e-19,  'NA',  0.00032381057438235483,  'potatoes, baked, flesh and skin'], ['herring,_flesh_only,_grilled_JCC',
          430.43478,  'NA',  0.0026762418646312765,  1.5518901361327963e-16,  1.173062731089902e-16,  'NA',
          0.0027918717888169345,  'herring, flesh only, grilled']]
    
        df_final = pd.DataFrame(data, columns = columns )
    return df_final